# Imports

## CV2

In [90]:
import cv2

This project uses basically the openCV library to make the facial recognition.
To know more you can acess the Github page of the project: https://github.com/opencv/opencv
There're many courses in the internet, (like Udemy), that explcains with a easy language the basics of the library

## Numpy

In [ ]:
import numpy as np

The numpy library make some calculus most faster than if they was made in Python, so is very recommended you to use

## OS

In [ ]:
import os

To open the photo files you need to use this library

## Multiprocessing

In [ ]:
from multiprocessing import TimeoutError
from multiprocessing.pool import ThreadPool

?To make a timed input we need to change some aspects of the input function

# Utils

## TimeOut

In [ ]:
def timeout(seconds):
    def decorator(function):
        def wrapper(*args, **kwargs):
            pool = ThreadPool(processes=1)
            result = pool.apply_async(function, args=args, kwds=kwargs)
            try:
                return result.get(timeout=seconds)
            except TimeoutError as e:
                return e
        return wrapper
    return decorator

This function alter the input time from infinite to the time specified by the user, (in this case 60s)

## Input functions

In [ ]:
@timeout(60)
def read_user_name(pergunta):
    return input(pergunta)

def verifica_resposta_valida(pergunta):
    while(1):
        resp = read_user_name(pergunta)
        if isinstance(resp, TimeoutError):
            resp = 'n'
        
        if(resp != 'S' and resp != 's' and resp != 'N' and resp != 'n'):
            print('Resposta Invalida! Tente Novamente!!!!!')
        else:
            break
    
    return resp.upper()
    

def insere_nome(pergunta = 'Nome: '):
    resp = read_user_name('Nome: ')
    if isinstance(resp, TimeoutError):
            resp = 'n'
    
    return resp.lower()

def insere_senha(pergunta = 'Senha: '):
    resp = read_user_name(pergunta)
    if isinstance(resp, TimeoutError):
            resp = 'n'
    
    return(resp.lower())

To use the timed input we have the "read username()" function that can receive a phrase to output or just use the standard phrase.

To apply some pre-determined filters in the valid answers we use the functions verifica_resposta_valida(), insere_nome() and insere_senha(), that, respectivly, only accepts 'S' or 'N' and convert all the input to lower case.

## Principal Modes Functions

In [ ]:
def modo_administrador(users, camera, classificador, classificadorolho):
    user = str()
    password = str()
    print('########## Modo Administrativo ##########')

    while(1):
        user = insere_nome('Usuario: ')
        password = insere_senha()
        usuarios = []
        senhas = []
        for x in users:
            usuarios.append(x[0])
            senhas.append(x[1])
        
        if(user in usuarios):
            index = usuarios.index(user)
            if(password == senhas[index]):
                break
            else:
                print('Usuario ou senha incorretas!!!!!')
                resp = verifica_resposta_valida('Deseja tentar novamente? [S/N]')
                if(resp == 'N'):
                    return 1
        else:
            print('Usuario ou senha incorretas!!!!!')
            resp = verifica_resposta_valida('Deseja tentar novamente? [S/N]')
            if(resp == 'N'):
                return 1
    
    while(1):
        print('#### Acesso Concedido ####')
        print('1. Criar novo usuário')
        print('2. Apagar Usuario')
        print('3. Listar Usuarios')
        print('4. Abrir Fechadura')
        print('5. Sair')
        q = input('Escolha uma das opcoes: ')

        if(q == '1'):
            while(1):
                user = insere_nome()
                
                if(user in usuarios):
                    print('Usuario já existe!!!!')
                    resp = verifica_resposta_valida('Deseja tentar novamente?[S/N] ')
                    if(resp == 'N'):
                        break
                else:
                    password = insere_senha()
                    confirmacao = insere_senha('Confirme a senha: ')
                    if(password == confirmacao):
                        novo_usuario(user, password, camera, classificador, classificadorolho)
                        break
                    else:
                        print('Senha e confirmacao nao batem!!!!!')
                        resp = verifica_resposta_valida('Deseja tentar novamente?[S/N] ')
                        if(resp == 'N'):
                            break

        elif(q == '2'):
            while(1):
                nome = insere_nome()
                
                if(nome not in usuarios):
                    print('Nome nao existe na base!!!!!!!!')
                    resp = verifica_resposta_valida('Deseja tentar novamente?[S/N] ')
                    if(resp == 'N'):
                        break
                else:
                    confirmacao = verifica_resposta_valida('Voce deseja realmente apagar o rosto de ' + nome + '?[S/N] ')
                    if(confirmacao == 'S'):
                        apagar_rosto(usuarios.find(nome))
                        break

        elif(q == '3'):
            print('Usuario, senha')
            for user in users:
                print(user)
                        
        elif(q == '4'):
            print('Porta aberta')
                
        else:
            break
    return 1

def modo_vigilancia(camera, classificador, classificadorolho):
    print('########## Modo Vigilância ##########')
    while(camera.isOpened()):
        conectado, imagem = camera.read() 
        if(conectado):
            imagemcinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
            facesdetectadas = classificador.detectMultiScale(imagemcinza, scaleFactor=1.5, minSize=(100,100))

        for (x,y,l,a) in facesdetectadas:
            cv2.rectangle(imagem, (x,y), (x+l, y+a), (0,0,255), 2)
        cv2.imshow('Face', imagem)

        #Botão para cancelar a execução
        key = cv2.waitKey(1) & 0xFF
        if (key==27):
            break
    camera.release()
    cv2.destroyAllWindows()

The programs functions basically with 2 principal modes, that are, the Administrator Mode and the Vigilance Mode.<br>
The Administrator Mode adds/remove users to the base, thats includ a User_name a Password and a series of photos to enable her/his passage. <br>
Other use of the Administrator Mode is to open the locker that is connected to Raspberry, this can be useful if the camera has any problem.<br>
The Vigilance mode just ability the camera and wait for someone to appear, and verify if the people that is in front of the door have permission to pass.

## Function to read the file users.txt

In [ ]:
def ler_users():
    arq = open('users.txt', 'r')
    users = arq.readlines()
    senhas = []
    for i in range(len(users)):
        if(i < len(users)-1):
            users[i] = users[i][:-1]
            usuario = users[i][:users[i].find(',')]
            password = users[i][1+users[i].find(','):]
            
            lista = [usuario, password]
            senhas.append(lista)
            
        else:
            usuario = users[i][:users[i].find(',')]
            password = users[i][1+users[i].find(','):]
            
            lista = [usuario, password]
            senhas.append(lista)

    return senhas

The principal file of users is basically a txt file that contains the user and the password of the user, separrated by commas and by space to separate each user.<br>
So the file is a thing like this:<br>
<br>
victor,placido<br>
sam,gomes<br>
root,root<br>
<br>
This function just read the file and return a list containing other lists that contain the username and the password of each user.<br>
Initially the function returns a dictionary of users like keys and password as values, but this proved to be a problem when we need to create a index to all the users. With the lists of users there position in the list is the index.<br>

## Catch the name of the IDs and Photos of all users

In [ ]:
def getimagemcomid(mostrar_ids = 0):
    caminhos = [os.path.join('fotos',f) for f in os.listdir('fotos')]
    faces=[]
    ids=[]
    for caminhoimagem in caminhos:
        imagemface = cv2.cvtColor(cv2.imread(caminhoimagem),cv2.COLOR_BGR2GRAY)
        id = int(os.path.split(caminhoimagem)[-1].split('.')[1])
        if(mostrar_ids == 1):
            print(id)
        ids.append(id)
        faces.append(imagemface)
    return np.array(ids), faces

This function catch all the IDs and photos needed by future functions to realize the training of the Raspberry Pi

## Function to take pictures and train the Raspberry

In [ ]:
def gravar_rosto(id, camera, classificador, classificadorolho):
    amostra = 1
    numeroamostras = 25
    largura = 200
    altura = 200
    print('Capturando as faces...')
    while(camera.isOpened()):
        conectado, imagem = camera.read()  
        if(conectado):
            imagemcinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
            print(np.average(imagemcinza))
            facesdetectadas = classificador.detectMultiScale(imagemcinza, scaleFactor=1.5, minSize=(150,150))

        for (x,y,l,a) in facesdetectadas:
            cv2.rectangle(imagem, (x,y), (x+l, y+a), (0,0,255), 2)
            regiao = imagem[y:y + a, x:x + l]
            regiaocinzaolho = cv2.cvtColor(regiao, cv2.COLOR_BGR2GRAY)

            olhosdetectados = classificadorolho.detectMultiScale(regiaocinzaolho)

            for(ox, oy, ol, oa) in olhosdetectados:
                cv2.rectangle(regiao, (ox, oy), (ox+ol, oy+oa), (0, 255, 0), 2)

                if(cv2.waitKey(1) & 0xFF == ord('q')):
                    if(np.average(imagemcinza) > 70):
                        imagemface = cv2.resize(imagemcinza[y:y + a, x:x + l], (largura, altura))
                        cv2.imwrite('fotos/pessoa.' + str(id) + '.' + str(amostra) + '.jpg', imagemface)
                        print('Foto ' + str(amostra) + ' capturada com sucesso!')
                        amostra = amostra+1
                            
        cv2.imshow('Face', imagem)
        cv2.waitKey(1)
        if(cv2.waitKey(1) & 0xFF == 27):
            break
        if(amostra >= numeroamostras + 1):
            break
    print('Faces capturadas om sucesso')
    camera.release()
    cv2.destroyAllWindows() 

    eigenface = cv2.face.EigenFaceRecognizer_create()
    fisherface = cv2.face.FisherFaceRecognizer_create()
    lbph = cv2.face.LBPHFaceRecognizer_create()
                  
    ids, faces = getimagemcomid(1) 
    print('Treinando...')

    eigenface.train(faces, ids)
    eigenface.write('classificadorEigen.yml')

    fisherface.train(faces, ids)
    fisherface.write('classificadorFisher.yml')

    lbph.train(faces, ids)
    lbph.write('classificadorLBPH.yml')

    print('Treinamento Realizado')
    return 1

This function take pictures with some conditions:<br>
    1-The light needs to be in a minimum of 110;<br>
    2-The raspberry must have recognized the eyes and face of the person.<br>
<br>
After the pictures has been captured the function realises the training of the Raspberry, using the three methods, (LBPH, Fisher and Eigen).

## Functions to create and remove a user

In [ ]:
def novo_usuario(novo_user, nova_pass, camera, classificador, classificadorolho):
    arq = open('users.txt', 'a')

    nova_linha = '\n' + novo_user + ',' + nova_pass
    arq.writelines(nova_linha)

    arq.close()
    
    users = ler_users()
    usuarios = []
    for x in users:
        usuarios.append(x[0])
        
    gravar_rosto(usuarios.index(user), camera, classificador, classificadorolho)
    
    print('Usuario ' + novo_user + ' adicionado!!!')
    return 1

def apagar_rosto(id_a_ser_removido):
    caminhos = [os.path.join('fotos',f) for f in os.listdir('fotos')]
    for caminhoimagem in caminhos:
        id = int(os.path.split(caminhoimagem)[-1].split('.')[1])
        if(id == id_a_ser_removido):
            os.remove(caminhoimagem)
    
    eigenface = cv2.face.EigenFaceRecognizer_create()
    fisherface = cv2.face.FisherFaceRecognizer_create()
    lbph = cv2.face.LBPHFaceRecognizer_create()
                      
    ids, faces = getimagemcomid(1) 
    print('Apagando...')
    
    eigenface.train(faces, ids)
    eigenface.write('classificadorEigen.yml')
    
    fisherface.train(faces, ids)
    fisherface.write('classificadorFisher.yml')
    
    lbph.train(faces, ids)
    lbph.write('classificadorLBPH.yml')
    
    print(str(id) + ' Apagado!!!')

This functions respectivly create a new user and remover a user.<br>
The user needs to input the name of the user that they want to create, a password and take the pictures required be the function "gravar_rosto()"
To remove a user the administrator need to input a name that are in the user.txt file

# Main

In [ ]:
classificador = cv2.CascadeClassifier("haarcascade-frontalface-default.xml")
classificadorolho = cv2.CascadeClassifier("haarcascade-eye.xml")
camera = cv2.VideoCapture(0)

users= ler_users()

resp = verifica_resposta_valida('Deseja entrar no modo administrador?[S/N] ')

if(resp == 'S'):
    modo_administrador(users, camera, classificador, classificadorolho)
    modo_vigilancia(camera, classificador, classificadorolho)
else:
    modo_vigilancia(camera, classificador, classificadorolho)

The main file of the program just receives the input of the user if he/she wants to enter administrator mode or the vigilance mode

# Compiled of all functions

In [2]:
def timeout(seconds):
    def decorator(function):
        def wrapper(*args, **kwargs):
            pool = ThreadPool(processes=1)
            result = pool.apply_async(function, args=args, kwds=kwargs)
            try:
                return result.get(timeout=seconds)
            except TimeoutError as e:
                return e
        return wrapper
    return decorator

@timeout(60)
def read_user_name(pergunta):
    return input(pergunta)

def verifica_resposta_valida(pergunta):
    while(1):
        resp = read_user_name(pergunta)
        if isinstance(resp, TimeoutError):
            resp = 'n'
        
        if(resp != 'S' and resp != 's' and resp != 'N' and resp != 'n'):
            print('Resposta Invalida! Tente Novamente!!!!!')
        else:
            break
    
    return resp.upper()
    

def insere_nome(pergunta = 'Nome: '):
    resp = read_user_name('Nome: ')
    if isinstance(resp, TimeoutError):
            resp = 'n'
    
    return resp.lower()

def insere_senha(pergunta = 'Senha: '):
    resp = read_user_name(pergunta)
    if isinstance(resp, TimeoutError):
            resp = 'n'
    
    return(resp.lower())


def modo_administrador(users, camera, classificador, classificadorolho):
    user = str()
    password = str()
    print('########## Modo Administrativo ##########')

    while(1):
        user = insere_nome('Usuario: ')
        password = insere_senha()
        usuarios = []
        senhas = []
        for x in users:
            usuarios.append(x[0])
            senhas.append(x[1])
        
        if(user in usuarios):
            index = usuarios.index(user)
            if(password == senhas[index]):
                break
            else:
                print('Usuario ou senha incorretas!!!!!')
                resp = verifica_resposta_valida('Deseja tentar novamente? [S/N]')
                if(resp == 'N'):
                    return 1
        else:
            print('Usuario ou senha incorretas!!!!!')
            resp = verifica_resposta_valida('Deseja tentar novamente? [S/N]')
            if(resp == 'N'):
                return 1
    
    while(1):
        print('#### Acesso Concedido ####')
        print('1. Criar novo usuário')
        print('2. Apagar Usuario')
        print('3. Listar Usuarios')
        print('4. Abrir Fechadura')
        print('5. Sair')
        q = input('Escolha uma das opcoes: ')

        if(q == '1'):
            while(1):
                user = insere_nome()
                
                if(user in usuarios):
                    print('Usuario já existe!!!!')
                    resp = verifica_resposta_valida('Deseja tentar novamente?[S/N] ')
                    if(resp == 'N'):
                        break
                else:
                    password = insere_senha()
                    confirmacao = insere_senha('Confirme a senha: ')
                    if(password == confirmacao):
                        novo_usuario(user, password, camera, classificador, classificadorolho)
                        break
                    else:
                        print('Senha e confirmacao nao batem!!!!!')
                        resp = verifica_resposta_valida('Deseja tentar novamente?[S/N] ')
                        if(resp == 'N'):
                            break

        elif(q == '2'):
            while(1):
                nome = insere_nome()
                
                if(nome not in usuarios):
                    print('Nome nao existe na base!!!!!!!!')
                    resp = verifica_resposta_valida('Deseja tentar novamente?[S/N] ')
                    if(resp == 'N'):
                        break
                else:
                    confirmacao = verifica_resposta_valida('Voce deseja realmente apagar o rosto de ' + nome + '?[S/N] ')
                    if(confirmacao == 'S'):
                        apagar_rosto(usuarios.find(nome))
                        break

        elif(q == '3'):
            print('Usuario, senha')
            for user in users:
                print(user)
                        
        elif(q == '4'):
            print('Porta aberta')
                
        else:
            break
    return 1

def modo_vigilancia(camera, classificador, classificadorolho):
    print('########## Modo Vigilância ##########')
    while(camera.isOpened()):
        conectado, imagem = camera.read() 
        if(conectado):
            imagemcinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
            facesdetectadas = classificador.detectMultiScale(imagemcinza, scaleFactor=1.5, minSize=(100,100))

        for (x,y,l,a) in facesdetectadas:
            cv2.rectangle(imagem, (x,y), (x+l, y+a), (0,0,255), 2)
        cv2.imshow('Face', imagem)

        #Botão para cancelar a execução
        key = cv2.waitKey(1) & 0xFF
        if (key==27):
            break
    camera.release()
    cv2.destroyAllWindows()


def ler_users():
    arq = open('users.txt', 'r')
    users = arq.readlines()
    senhas = []
    for i in range(len(users)):
        if(i < len(users)-1):
            users[i] = users[i][:-1]
            usuario = users[i][:users[i].find(',')]
            password = users[i][1+users[i].find(','):]
            
            lista = [usuario, password]
            senhas.append(lista)
            
        else:
            usuario = users[i][:users[i].find(',')]
            password = users[i][1+users[i].find(','):]
            
            lista = [usuario, password]
            senhas.append(lista)

    return senhas


def novo_usuario(novo_user, nova_pass, camera, classificador, classificadorolho):
    arq = open('users.txt', 'a')

    nova_linha = '\n' + novo_user + ',' + nova_pass
    arq.writelines(nova_linha)

    arq.close()
    
    users = ler_users()
    usuarios = []
    for x in users:
        usuarios.append(x[0])
        
    gravar_rosto(usuarios.index(user), camera, classificador, classificadorolho)
    
    print('Usuario ' + novo_user + ' adicionado!!!')
    return 1

def getimagemcomid(mostrar_ids = 0):
    caminhos = [os.path.join('fotos',f) for f in os.listdir('fotos')]
    faces=[]
    ids=[]
    for caminhoimagem in caminhos:
        imagemface = cv2.cvtColor(cv2.imread(caminhoimagem),cv2.COLOR_BGR2GRAY)
        id = int(os.path.split(caminhoimagem)[-1].split('.')[1])
        if(mostrar_ids == 1):
            print(id)
        ids.append(id)
        faces.append(imagemface)
    return np.array(ids), faces

def apagar_rosto(id_a_ser_removido):
    caminhos = [os.path.join('fotos',f) for f in os.listdir('fotos')]
    for caminhoimagem in caminhos:
        id = int(os.path.split(caminhoimagem)[-1].split('.')[1])
        if(id == id_a_ser_removido):
            os.remove(caminhoimagem)
    
    eigenface = cv2.face.EigenFaceRecognizer_create()
    fisherface = cv2.face.FisherFaceRecognizer_create()
    lbph = cv2.face.LBPHFaceRecognizer_create()
                      
    ids, faces = getimagemcomid(1) 
    print('Apagando...')
    
    eigenface.train(faces, ids)
    eigenface.write('classificadorEigen.yml')
    
    fisherface.train(faces, ids)
    fisherface.write('classificadorFisher.yml')
    
    lbph.train(faces, ids)
    lbph.write('classificadorLBPH.yml')
    
    print(str(id) + ' Apagado!!!')

def gravar_rosto(id, camera, classificador, classificadorolho):
    amostra = 1
    numeroamostras = 25
    largura = 200
    altura = 200
    print('Capturando as faces...')
    while(camera.isOpened()):
        conectado, imagem = camera.read()  
        if(conectado):
            imagemcinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
            print(np.average(imagemcinza))
            facesdetectadas = classificador.detectMultiScale(imagemcinza, scaleFactor=1.5, minSize=(150,150))

        for (x,y,l,a) in facesdetectadas:
            cv2.rectangle(imagem, (x,y), (x+l, y+a), (0,0,255), 2)
            regiao = imagem[y:y + a, x:x + l]
            regiaocinzaolho = cv2.cvtColor(regiao, cv2.COLOR_BGR2GRAY)

            olhosdetectados = classificadorolho.detectMultiScale(regiaocinzaolho)

            for(ox, oy, ol, oa) in olhosdetectados:
                cv2.rectangle(regiao, (ox, oy), (ox+ol, oy+oa), (0, 255, 0), 2)

                if(cv2.waitKey(1) & 0xFF == ord('q')):
                    if(np.average(imagemcinza) > 70):
                        imagemface = cv2.resize(imagemcinza[y:y + a, x:x + l], (largura, altura))
                        cv2.imwrite('fotos/pessoa.' + str(id) + '.' + str(amostra) + '.jpg', imagemface)
                        print('Foto ' + str(amostra) + ' capturada com sucesso!')
                        amostra = amostra+1
                            
        cv2.imshow('Face', imagem)
        cv2.waitKey(1)
        if(cv2.waitKey(1) & 0xFF == 27):
            break
        if(amostra >= numeroamostras + 1):
            break
    print('Faces capturadas om sucesso')
    camera.release()
    cv2.destroyAllWindows() 

    eigenface = cv2.face.EigenFaceRecognizer_create()
    fisherface = cv2.face.FisherFaceRecognizer_create()
    lbph = cv2.face.LBPHFaceRecognizer_create()
                  
    ids, faces = getimagemcomid(1) 
    print('Treinando...')

    eigenface.train(faces, ids)
    eigenface.write('classificadorEigen.yml')

    fisherface.train(faces, ids)
    fisherface.write('classificadorFisher.yml')

    lbph.train(faces, ids)
    lbph.write('classificadorLBPH.yml')

    print('Treinamento Realizado')
    return 1